In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.feature_selection import SelectPercentile,SelectKBest

In [3]:
from sklearn.linear_model import LinearRegression,LogisticRegression
from sklearn.feature_selection import SelectFromModel

In [4]:
titanic=sns.load_dataset('titanic')

In [5]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [6]:
titanic.isnull().sum()

survived         0
pclass           0
sex              0
age            177
sibsp            0
parch            0
fare             0
embarked         2
class            0
who              0
adult_male       0
deck           688
embark_town      2
alive            0
alone            0
dtype: int64

In [7]:
titanic.drop(labels=['age','deck'],axis=1,inplace=True)
titanic=titanic.dropna() #embark town 에서 정보 없는 2명 데이터 없어짐
data=titanic[['pclass','sex','sibsp','parch','embarked','who','alone']].copy()

In [8]:
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,male,1,0,S,man,False
1,1,female,1,0,C,woman,False
2,3,female,0,0,S,woman,True
3,1,female,1,0,S,woman,False
4,3,male,0,0,S,man,True


In [9]:
sex={'male':0,'female':1}
data['sex']=data['sex'].map(sex)

embark={'S':0,'C':1,'Q':2}
data['embarked']=data['embarked'].map(embark)

who={'man':0,'woman':1,'child':2}
data['who']=data['who'].map(who)

alone={True:1,False:0}
data['alone']=data['alone'].map(alone)

In [10]:
data.head()

,pclass,sex,sibsp,parch,embarked,who,alone
0,3,0,1,0,0,0,0
1,1,1,1,0,1,1,0
2,3,1,0,0,0,1,1
3,1,1,1,0,0,1,0
4,3,0,0,0,0,0,1


In [11]:
x=data.copy()
y=titanic['survived']

In [12]:
x.shape , y.shape

((889, 7), (889,))

In [15]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=42)

### Estimation of coefficients of Linear Regression

In [16]:
sel=SelectFromModel(LinearRegression())
#selectfrommodel : weight 높은것만 가져옴. threshold가 default면 평균보다 높은것 가져옴

In [17]:
sel.fit(x_train,y_train)

SelectFromModel(estimator=LinearRegression())

In [18]:
sel.get_support()

array([ True,  True, False, False, False,  True, False])

In [19]:
sel.estimator_.coef_

array([-0.14993774,  0.29999614, -0.08502325, -0.05323123,  0.03540811,
        0.23022328, -0.1143598 ])

In [22]:
features=x_train.columns[sel.get_support()]
features

Index(['pclass', 'sex', 'who'], dtype='object')

In [29]:
x_train_reg=sel.transform(x_train)
x_test_reg=sel.transform(x_test)

In [31]:
x_train_reg.shape , x_test_reg.shape

((622, 3), (267, 3))

In [32]:
def run_randomforest(x_train,x_test,y_train,y_test):
    clf=RandomForestClassifier(n_estimators=100,random_state=0,n_jobs=-1)
    clf.fit(x_train,y_train)
    y_pred=clf.predict(x_test)
    print('Accuracy:',accuracy_score(y_test,y_pred))

In [33]:
%%time
run_randomforest(x_train_reg,x_test_reg,y_train,y_test)

Accuracy: 0.797752808988764
Wall time: 254 ms


In [34]:
%%time
run_randomforest(x_train,x_test,y_train,y_test)

Accuracy: 0.8052434456928839
Wall time: 355 ms


### Logistic Regression Coefficient with L1 Regularization

In [41]:
sel=SelectFromModel(LogisticRegression(penalty='l1',C=0.05,solver='liblinear')) #C값 줄여나가면서 보면 됨
sel.fit(x_train,y_train)
sel.get_support()

array([ True,  True,  True, False, False,  True, False])

In [42]:
sel.estimator_.coef_

array([[-0.54238401,  0.97242217, -0.18504656,  0.        ,  0.        ,
         0.90405789,  0.        ]])

In [44]:
x_train_l1=sel.transform(x_train)
x_test_l1=sel.transform(x_test)

In [45]:
%%time
run_randomforest(x_train_l1,x_test_l1,y_train,y_test)

Accuracy: 0.8239700374531835
Wall time: 248 ms


### Logistic Regression Coefficient with L2 Regularization

In [50]:
sel=SelectFromModel(LogisticRegression(penalty='l2',C=0.05,solver='liblinear')) #C값 줄여나가면서 보면 됨
sel.fit(x_train,y_train)
sel.get_support()

array([ True,  True, False, False, False,  True, False])

In [51]:
sel.estimator_.coef_

array([[-0.59213357,  0.93595438, -0.36199847, -0.06897201,  0.21033721,
         1.00605131, -0.23906564]])

In [52]:
x_train_l2=sel.transform(x_train)
x_test_l2=sel.transform(x_test)

In [53]:
%%time
run_randomforest(x_train_l2,x_test_l2,y_train,y_test)

Accuracy: 0.797752808988764
Wall time: 253 ms
